# Cleaning Data at Scale

By Dr. Phil Winder of https://WinderResearch.com

Hi there! Welcome to this notebook which was originally written to be used with the training of the same name. If you like this then please visit my website for more, [tweet @DrPhilWinder](https://twitter.com/DrPhilWinder) or get in touch via [Linkedin](https://www.linkedin.com/in/DrPhilWinder/) or plain old [email](mailto:phil@WinderResearch.com).

Table of Contents:

1. [Visualising Data](#1.-Visualising-Data)
2. [Fixing Missing Data](#2.-Fixing-Missing-Data)
3. [Detect and Remove Outliers](#3.-Detect-and-Remove-Outliers)
4. [Is it Normal?](#4.-Is-it-Normal?)
5. [Fixing Non-Normal Data](#5.-Fixing-Non-Normal-Data)
6. [Fixing Scales and Categorical Data](#6:-Fixing-Scales-and-Categorical-Data)
7. [Model Improvement through Feature Selection](#7:-Model-Improvement-through-Feature-Selection)
8. [Cleaning Timeseries Data](#8:-Cleaning-Timeseries-Data)

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns

In [ ]:
sns.set(style="white")
matplotlib.rc('figure', figsize=[12, 5])

# 1. Visualising Data

This first section is all about visualising your data. In my opinion, manually visualising data is the most important Data Science technique, but also the most underrepresented.

Below I concentrate on demonstrating some techniques that will help you pre-process and clean your data.

In [ ]:
!pip install missingno > /dev/null
import pandas as pd
import missingno as msno
import numpy as np

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)

In [ ]:
titanic.hist(color='dimgray', layout=(2, 4));

In [ ]:
msno.matrix(titanic.sample(500));

In [ ]:
msno.bar(titanic.sample(500));

In [ ]:
msno.heatmap(titanic);

In [ ]:
pd.plotting.scatter_matrix(titanic, color='dimgray');

In [ ]:
# Compute the correlation matrix
corr = titanic.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=0.8, vmin=-0.8, square=True);

In [ ]:
url = "../data/nypd-motor-vehicle-collisions.csv"
collisions = pd.read_csv(url)

---
## Challenge

Above is the NYPD collisions dataset that reports the causes of traffic incidents in New York (not to be confused with York) in the USA.

- Try plotting some visualisations of this data
- What can you tell me about it?

---

# 2. Fixing Missing Data

Most real world datasets have missing values. Many examples online and in the literature present data that has already been cleaned. But in industry, significant amounts of data are missing. Let's get some experience working with that.

In [ ]:
import pandas as pd
import sklearn.linear_model

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)

In [ ]:
titanic = titanic[['survived', 'fare', 'age']]
titanic.count()

Pandas has a very handy function called `dropna()` that removes all rows that have at least one `NaN`.

In [ ]:
titanic.dropna().count()

We can also replace missing values with another value. Zero, for example:

In [ ]:
titanic.fillna(0).describe()

Or with some function of the columns, like the mean or the median:

In [ ]:
titanic.fillna(titanic.mean()).describe()

Note how the `age` feature changes quite dramatically depending on what you impute it with. You will probably want to plot the histogram of this in real life to make sure you're not trashing the statistics.

We could go further and even create a regression model to predict the ages...

In [ ]:
titanic_dropped = titanic.dropna()
X = titanic_dropped[['survived', 'fare']]
y = titanic_dropped['age']
mdl = sklearn.linear_model.LinearRegression().fit(X, y)

In [ ]:
titanic_X = titanic.dropna(subset = ['survived', 'fare']) # Inputs to an sklearn model can't have NaNs
print(titanic_X.count())
titanic_imputed = titanic_X.copy()
titanic_imputed['age_imputed'] = mdl.predict(titanic_X[['survived', 'fare']])
print(titanic_imputed.tail(10))
print(titanic_imputed.count())
titanic_fixed = titanic_X.copy()
titanic_fixed['age'] = titanic_fixed['age'].fillna(titanic_imputed['age_imputed'])
print(titanic_fixed.tail(10))
print(titanic_fixed.count())

Beware, every time we impute the data we're adding subtle biases. For example, if we imputed one feature that was mostly `NaN`s the effect would be that this feature correlates with other features that produced it. If we were performing a classification task, this would be pointless.

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)
titanic.head()

In [ ]:
url = "../data/property_data.csv"
housing = pd.read_csv(url)
housing = housing[['NUM_BATH', 'SQ_FT', 'OWN_OCCUPIED']]
housing = housing.replace(['--', 'HURLEY', 'na', '12'], np.nan)
housing = housing.replace(['Y'], 1)
housing = housing.replace(['N'], 0)
housing

---
## Challenge

There is a very simple hacky dataset above.

1. Try dropping `NaN` rows?
2. What about replacing with zeros?
3. Try to impute using a regression model

---

# 3. Detect and Remove Outliers

Outliers are tricky because we often don't have any ground truth to prove that outliers truly are outliers. So largely this is part art.

Here's an example.

In [ ]:
arr = pd.DataFrame([10, 386, 479, 627, 20, 523, 482, 483, 542, 699, 535, 617, 577, 471, 615, 583, 441, 562, 563, 527, 453, 530, 433, 541, 585, 704, 443, 569, 430, 637, 331, 511, 552, 496, 484, 566, 554, 472, 335, 440, 579, 341, 545, 615, 548, 604, 439, 556, 442, 461, 624, 611, 444, 578, 405, 487, 490, 496, 398, 512, 422, 455, 449, 432, 607, 679, 434, 597, 639, 565, 415, 486, 668, 414, 665, 763, 557, 304, 404, 454, 689, 610, 483, 441, 657, 590, 492, 476, 437, 483, 529, 363, 711, 543])
arr = (arr - arr.mean()) / arr.std()
arr.hist();

In [ ]:
arr = arr[np.abs(arr) < 3]
arr.hist();

In [ ]:
url = "../data/winequality-red.csv"
wine = pd.read_csv(url)
wine = pd.read_csv(url, sep=";")
wine = wine[['alcohol', 'quality']]
wine.head(n=5)

---
## Challenge

Above is a more realistic wine dataset. Can you remove the outliers?

---

# 4. Is it Normal?

Normality is important for a range of algorithms. Here we investigate, what is normal? 

In [ ]:
import scipy
import matplotlib.pyplot as plt

In [ ]:
url = "../data/starsCYG.csv"
starsCYG = pd.read_csv(url)
var = 'log.Te'

ax = sns.distplot(starsCYG[var], kde=False, norm_hist=False)
ax.set(title='Star Cluster CYG OB1', xlabel='log(Temperature)', ylabel='Count');

In [ ]:
scipy.stats.probplot(
    starsCYG[var], plot=sns.mpl.pyplot)
ax = plt.gca()
ax.get_lines()[0].set_markerfacecolor(sns.color_palette()[0])
ax.get_lines()[0].set_markeredgewidth(0)
ax.get_lines()[1].set_color(sns.color_palette()[1])
ax.set(title='Normal Quantile-Quantile Plot',
       xlabel='Theoretical Normal Quantiles', ylabel='Ordered Data');

In [ ]:
# Distribution fitting

y = starsCYG[var]
x_plot = scipy.linspace(min(y) - 1, max(y) + 1, 100)
y_count, x = np.histogram(y, density=True)
x = (x + np.roll(x, -1))[:-1] / 2.0
h = sns.distplot(y, kde=False, norm_hist=True)

dist_names = ['gamma', 'beta', 'rayleigh', 'norm', 'pareto', 'powernorm']

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    pdf_fitted = dist.pdf(x, loc=loc, scale=scale, *arg)
    sse = np.sum(np.power(y_count - pdf_fitted, 2.0))
    pdf_plot = dist.pdf(x_plot, loc=loc, scale=scale, *arg)
    sns.lineplot(x_plot, pdf_plot, label='{} ({:.1f})'.format(dist_name, sse))
plt.legend(loc='upper right')
ax = plt.gca()
ax.set(title='Fitted Distributions (with SSE scores)',
       xlabel='log(Temperature)', ylabel='Normalised Count');

In [ ]:
data = pd.DataFrame(2 * np.random.randn(200) + 20)

---
## Challenge

Using the data `data`, is it normally distributed?

---

# 5. Fixing Non-Normal Data

Once you know that you have some data that is not normal, you can attempt to bring it back to normality.

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)
fare = titanic['fare']

In [ ]:
fare.hist();

---
## Challenge

Try applying a few mathematical functions to invert this data. Have a go using `np.log`, `np.power`, `np.sqrt`.

---

# 6: Fixing Scales and Categorical Data

Fixing the scales of data is important to make it as easy as possible for the model. This is a classification example - where I limit the number of training iterations to make the point - that breaks because the scales are so skewed. After min-max scaling the data, the model finds it much easier to iterate towards the result.

In [ ]:
np.random.seed(42)
url = "../data/titanic.csv"
titanic = pd.read_csv(url)
titanic = titanic[['survived', 'fare', 'age']]
titanic.dropna(inplace=True)
titanic = titanic[(titanic[['fare', 'age']] != 0).all(axis=1)]  # Remove zero fares
y = titanic['survived']
X = titanic[['fare', 'age']]

clf = sklearn.svm.LinearSVC(tol=1e-2, max_iter=10).fit(X, y)
xx, yy = np.mgrid[0:600:1, 0:100:1]
grid = np.c_[xx.ravel(), yy.ravel()]
probs = clf.predict(grid).reshape(xx.shape)
score = clf.score(X, y) * 100

fig, (ax1, ax2) = plt.subplots(ncols=2)

sns.scatterplot(data=X, x='fare', y='age', hue=y, linewidth=0, ax=ax1).set_title(
    "Titanic Survivors - SVM (10 iter) - {:.1f}%".format(score))
ax1.contour(xx, yy, probs, levels=[.5], cmap="Greys", vmin=0, vmax=.6)

X[:] = sklearn.preprocessing.MinMaxScaler().fit_transform(X[:])
clf = sklearn.svm.LinearSVC(tol=1e-2, max_iter=10).fit(X, y)
xx, yy = np.mgrid[0:1:0.001, 0:1:0.001]
grid = np.c_[xx.ravel(), yy.ravel()]
probs = clf.predict(grid).reshape(xx.shape)
score = clf.score(X, y) * 100

sns.scatterplot(data=X, x='fare', y='age', hue=y, linewidth=0, ax=ax2).set_title(
    "Titanic Survivors - SVM (10 iter, scaled) - {:.1f}%".format(score))
ax2.contour(xx, yy, probs, levels=[.5], cmap="Greys", vmin=0, vmax=.6);

In [ ]:
s = pd.Series(list('abcaba'))
pd.get_dummies(s)

Above we're rencoding categorical variables into new features. Now we can pass this data into our standard models.

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)

In [ ]:
X = titanic[['age', 'embarked', 'fare', 'survived']]
X = X.dropna()
y = X[['survived']]
X = X[['age', 'embarked', 'fare']]

---
## Challenge

Above we see the Titanic data again. 

- Try to recode the `embarked` feature
- Then use the features X to predict the label y in a classification model

---

# 7: Model Improvement through Feature Selection

The Decision Tree classifier (and variants of) attempts to segment the data into "pure" buckets via simple thresholds. The split that produces the most "pure" bucket is deemed to be a good split.

We can use this definition of "good" to provide some information about how well a single feature is able to split segment the data according to the labels. "Better" features will have a higher score.

The example below uses the titanic dataset again to demonstrate this.

In [ ]:
import pandas as pd
import sklearn.tree
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "../data/titanic.csv"
titanic = pd.read_csv(url)
X = titanic[['age', 'fare', 'survived']]
X = X.dropna()
y = X[['survived']]
X = X[['age', 'fare']]

In [ ]:
mdl = sklearn.tree.DecisionTreeClassifier().fit(X, y)

In [ ]:
plt.bar(X.columns, mdl.feature_importances_)
plt.gca().set_ylabel('Relative importance');

We can see that the `fare` feature is more informative than the `age` parameter.

### Brute Force

Another thing we can do is iterate over features to find the best combination. Let's use `mlxtend` to implement this for us (and borrow an example)

In [ ]:
!pip install mlxtend > /dev/null

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.data import wine_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X, y = wine_data()
X_train, X_test, y_train, y_test= train_test_split(X, y, 
                                                   stratify=y,
                                                   test_size=0.3,
                                                   random_state=1)

knn = KNeighborsClassifier(n_neighbors=2)

sfs1 = SFS(estimator=knn, 
           k_features=(3, 10),
           forward=True, 
           floating=False, 
           scoring='accuracy',
           cv=5)

pipe = make_pipeline(StandardScaler(), sfs1)

pipe.fit(X_train, y_train)

print('best combination (ACC: %.3f): %s\n' % (sfs1.k_score_, sfs1.k_feature_idx_))
# print('all subsets:\n', sfs1.subsets_)
plot_sfs(sfs1.get_metric_dict(), kind='std_err');

In [ ]:
from sklearn.datasets import load_iris
from mlxtend.evaluate import PredefinedHoldoutSplit
import numpy as np


iris = load_iris()
X = iris.data
y = iris.target

---
## Challenge

Above is the infamous iris dataset.

- Can you tell me which feature is the most informative?

---

# 8: Cleaning Timeseries Data

Your domain might involve timeseries data. Much of what we have discussed also applies to the time domain. However there is a significant amount of knowledge that has been accumulated under the specialisation of Signal Processing.

But just to give you an idea, lets use the library `statsmodels` to give you some idea.

In [ ]:
url = "../data/airline-passengers.csv"
airline = pd.read_csv(url)
airline.columns = ['month', 'passengers']
airline['month'] = pd.to_datetime(airline['month'])
airline.set_index('month', inplace=True)
airline.index = pd.DatetimeIndex(airline.index.values,freq=airline.index.inferred_freq)

In [ ]:
airline.plot();

First let me show you how to do some exponential smoothing. This is a simple averaging technique to perform "noise reduction". Note that we're not really reducing any noise, we're just saying that the rolling average of the data is a better representation of that data.

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing

airline.plot()

# Simple Exponential Smoothing
for alpha in [0.3, 0.1]:
    fit = SimpleExpSmoothing(airline).fit(smoothing_level=alpha,optimized=False)
    fit.fittedvalues.rename(r'$\alpha={}$'.format(alpha)).plot()
plt.legend();

Now let's remove a few samples to simulate some `NaN`s. Then we can perform some interpolation to fix it.

In [ ]:
airline_corrupt = airline.copy()
airline_corrupt.iloc[10:11] = np.nan
airline_corrupt.iloc[30:35] = np.nan
airline_corrupt.iloc[60:100] = np.nan

In [ ]:
airline_corrupt.plot();

In [ ]:
airline_corrupt.interpolate().plot();

Oneliner!

Note how pandas in using a linear implementation of interpolation. So we lose a lot of the seasonality in the data. 

Also note that the bigger the gap, the worse it seems to perform. We can barely see the single `NaN` around position 10.

Still very good for such little code though!

### Bonus: Seasonal decomposition

I thought I'd just add this as it's related. If you have seasonal data, `statsmodels` does a really nice job of decomposing the data to show you the periodicity and trend in the data. Take a look below:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(airline, model='multiplicative')
result.plot();

In [ ]:
airline_outliers = airline.copy()
airline_outliers.iloc[10:11] = 0
airline_outliers.iloc[30:35] = 10000
airline_outliers.iloc[60:100] = 0

---
## Challenge

The dataset above contains outliers.

- Can you find the outliers?
- Can you remove the outliers and replace with an interpolation?

---